In [36]:
import json
import spacy
import re
import difflib as dl
file_path = '/content/drive/MyDrive/NLP/gg2013.json'

#source1=https://catriscode.com/2021/03/02/extracting-or-removing-mentions-and-hashtags-in-tweets-using-python/
#source2== https://stackoverflow.com/questions/38365389/compare-similarity-between-names/38365913

In [37]:
import re, math
from collections import Counter

WORD = re.compile(r'\w+')

def get_cosine(vec1, vec2):
    # print vec1, vec2
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x]**2 for x in vec1.keys()])
    sum2 = sum([vec2[x]**2 for x in vec2.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)

    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

def text_to_vector(text):
    return Counter(WORD.findall(text))

def get_similarity(a, b):
    a = text_to_vector(a.strip().lower())
    b = text_to_vector(b.strip().lower())

    return get_cosine(a, b)

In [38]:
f = open(file_path)
data = json.load(f)
nlp = spacy.load("en_core_web_sm")
tweet_texts = []

for tweet in data:
  cleaned_tweet = re.sub('#([a-zA-Z0-9_]{1,50})', '', tweet['text'])
  cleaned_tweet = re.sub('@([a-zA-Z0-9_]{1,50})', '', cleaned_tweet)
  cleaned_tweet = cleaned_tweet.lower()
  if 'rt:' not in cleaned_tweet:
    tweet_texts.append(cleaned_tweet)

In [39]:
host_key_words ='will be hosting|are hosting|is hosting|hosted'
hosts = {}
for tweet_text in tweet_texts:
  if re.search(host_key_words,tweet_text):
    doc = nlp(tweet_text)
    for entity in doc.ents:
      if entity.label_ == 'PERSON':
        if entity.text in hosts.keys():
          hosts[entity.text] = hosts[entity.text] + 1
        else:
          hosts[entity.text] = 1

In [40]:
hosts = dict(sorted(hosts.items(),key = lambda host:host[1]))

In [41]:
host_names = hosts.keys()
for host in list(hosts.keys()):
  for small_host in list(hosts.keys()):
    if small_host!=host and hosts[small_host]<hosts[host] and get_similarity(host,small_host)>0.7:
      if (len(host)==1 and len(small_host)>1):
        hosts[small_host] = hosts[host] + hosts[small_host]
        hosts.pop(host, None)
        break
      else:
        hosts[host] = hosts[host] + hosts[small_host]
        hosts.pop(small_host, None)

In [42]:
actual_hosts = []
max = 0
for host in hosts.keys():
  if hosts[host] >=max:
    max = hosts[host]

for host in hosts.keys():
  if hosts[host]/max >=0.4:
    actual_hosts.append(host)
    print(host+':'+str(hosts[host]/max))

print(actual_hosts)

tina:0.42424242424242425
amy poehler:1.0
['tina', 'amy poehler']
